In [1]:
import pickle
from keras.models import load_model
from pydub import AudioSegment
import numpy as np
#from vote_classifier import AudioPredictor
#from vote_classifier import VoteClassifier

import librosa

C:\Users\ssttg\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [7]:
with open('audio_classifier_model.pkl', 'rb') as file:
    audio_model = pickle.load(file)

C:\Users\ssttg\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:719: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [9]:
audio_model.audio_classify('./TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step


'3'

In [5]:
# Step 2: Define the emotion labels (should match what your model was trained on)
emotion_labels = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]

# Step 3: Create an instance of the AudioPredictor class
predictor = AudioPredictor('my_lstm_model_88.h5', emotion_labels)

# Step 4: Specify the path to your audio file
audio_file_path = './TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav'  # Replace with your actual audio file path

# Step 5: Call the `predict()` method to get the emotion prediction
predicted_emotion = predictor.predict(audio_file_path)

# Step 6: Print the result
print("Predicted Emotion: ", predicted_emotion)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 278ms/step
Predicted Emotion:  Anger


In [7]:
# Step 2: Define the emotion labels (should match what your model was trained on)
emotion_labels = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]

# Step 3: Create an instance of the AudioPredictor class
predictor = AudioPredictor(audio_model)

# Step 4: Specify the path to your audio file
audio_file_path = './TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav'  # Replace with your actual audio file path

# Step 5: Call the `predict()` method to get the emotion prediction
predicted_emotion = predictor.audio_classify(audio_file_path)

predicted_index = int(predicted_emotion)

# Access the corresponding emotion label using the index
if 0 <= predicted_index < len(emotion_labels):
    corresponding_emotion = emotion_labels[predicted_index]
    print(f"Predicted emotion (index {predicted_index}): {corresponding_emotion}")
else:
    print(f"Index {predicted_index} is out of range for emotion_labels.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
Predicted emotion (index 3): Anger


In [5]:
class AudioPredictor:
    def __init__(self, model):
        # Load the model directly
        self.model = model

    def preprocess_audio(self, audio_path):
        raw_audio = AudioSegment.from_file(audio_path)
        samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
        
        # Trim silence from the beginning and end
        trimmed, _ = librosa.effects.trim(samples, top_db=25)

        # Pad to a fixed length if necessary (e.g., 180,000 samples)
        padded = np.pad(trimmed, (0, max(0, 180000 - len(trimmed))), 'constant')
        sr = raw_audio.frame_rate  # Get the sample rate

        return padded, sr

    def extract_features(self, audio, sr):
        # Extract features similar to training
        zcr = librosa.feature.zero_crossing_rate(audio)
        rms = librosa.feature.rms(y=audio)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)

        # Concatenate features to create the input array
        features = np.concatenate((
            np.swapaxes(zcr, 0, 1), 
            np.swapaxes(rms, 0, 1), 
            np.swapaxes(mfccs, 0, 1)), 
            axis=1
        )

        # Reshape for LSTM input (1 sample, timesteps, features)
        features = features.reshape(1, features.shape[0], features.shape[1])  # Add batch dimension
        return features

    def audio_classify(self, audio_path):
        # Preprocess the audio file and extract features
        processed_audio, sr = self.preprocess_audio(audio_path)
        features = self.extract_features(processed_audio, sr)

        # Make predictions using the loaded model
        predictions = self.model.predict(features)

        # Get the predicted class
        predicted_class = np.argmax(predictions, axis=1)
        return str(predicted_class[0])


In [9]:
with open('audio_classifier_model.pkl', 'wb') as file:
    pickle.dump(predictor, file)


In [3]:
# Load the audio model (e.g., an LSTM model for audio)
audio_model = load_model('my_lstm_model_88.h5')

In [26]:
# Function to preprocess the audio
def preprocess_audio(path):
    raw_audio = AudioSegment.from_file(path)
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    
    # Trim silence from the beginning and end
    trimmed, _ = librosa.effects.trim(samples, top_db=25)

    # Pad to a fixed length if necessary
    padded = np.pad(trimmed, (0, max(0, 180000 - len(trimmed))), 'constant')
    sr = raw_audio.frame_rate  # Get sample rate

    return padded, sr


In [28]:
# Function to make a prediction
def predict_audio(model, audio_path):
    # Preprocess the audio file
    processed_audio, sr = preprocess_audio(audio_path)

    # Extract features similar to training
    zcr = librosa.feature.zero_crossing_rate(processed_audio)
    rms = librosa.feature.rms(y=processed_audio)
    mfccs = librosa.feature.mfcc(y=processed_audio, sr=sr, n_mfcc=13)

    # Concatenate features to create the input array
    features = np.concatenate((
        np.swapaxes(zcr, 0, 1), 
        np.swapaxes(rms, 0, 1), 
        np.swapaxes(mfccs, 0, 1)), 
        axis=1
    )

    # Reshape for LSTM input (1 sample, timesteps, features)
    features = features.reshape(1, features.shape[0], features.shape[1])  # Add batch dimension

    # Make predictions
    predictions = model.predict(features)

    # Get the predicted class
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class


In [40]:
# Example usage
audio_file_path = './TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav'  # Replace with your audio file path
predicted_emotion = predict_audio(audio_model, audio_file_path)

# Assuming you have a mapping of class indices to emotion labels
emotion_labels = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]

predicted_emotion_label = emotion_labels[predicted_emotion[0]]

print("Predicted Emotion: ", predicted_emotion_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 319ms/step
Predicted Emotion:  Anger


In [1]:
import numpy as np
import librosa
from pydub import AudioSegment
from keras.models import load_model

# Load the audio model (e.g., an LSTM model for audio)
audio_model = load_model('my_lstm_model_88.h5')

# Function to preprocess the audio
def preprocess_audio(path):
    raw_audio = AudioSegment.from_file(path)
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    
    # Trim silence from the beginning and end
    trimmed, _ = librosa.effects.trim(samples, top_db=25)

    # Pad to a fixed length if necessary
    padded = np.pad(trimmed, (0, max(0, 180000 - len(trimmed))), 'constant')
    sr = raw_audio.frame_rate  # Get sample rate

    return padded, sr

# Function to make a prediction
def predict_audio(model, audio_path):
    # Preprocess the audio file
    processed_audio, sr = preprocess_audio(audio_path)

    # Extract features similar to training
    zcr = librosa.feature.zero_crossing_rate(processed_audio)
    rms = librosa.feature.rms(y=processed_audio)
    mfccs = librosa.feature.mfcc(y=processed_audio, sr=sr, n_mfcc=13)

    # Concatenate features to create the input array
    features = np.concatenate((
        np.swapaxes(zcr, 0, 1), 
        np.swapaxes(rms, 0, 1), 
        np.swapaxes(mfccs, 0, 1)), 
        axis=1
    )

    # Reshape for LSTM input (1 sample, timesteps, features)
    features = features.reshape(1, features.shape[0], features.shape[1])  # Add batch dimension

    # Make predictions
    predictions = model.predict(features)

    # Get the predicted class
    predicted_class = np.argmax(predictions, axis=1)

    return predicted_class

# Example usage
audio_file_path = './TESS Toronto emotional speech set data/OAF_angry/OAF_bath_angry.wav'  # Replace with your audio file path
predicted_emotion = predict_audio(audio_model, audio_file_path)

# Assuming you have a mapping of class indices to emotion labels
emotion_labels = ["Happiness", "Neutral", "Sadness", "Anger", "Fear", "Disgust"]

predicted_emotion_label = emotion_labels[predicted_emotion[0]]

print("Predicted Emotion: ", predicted_emotion_label)


C:\Users\ssttg\anaconda3\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 310ms/step
Predicted Emotion:  Anger
